# 04 - Feature Engineering - Violência no Rio de Janeiro

Este notebook realiza feature engineering completo para preparar dados para modelagem de machine learning, incluindo:

## Objetivos do Feature Engineering

1. **Features Temporais**: Lags, médias móveis, sazonalidade
2. **Features Espaciais**: Distâncias, densidades, vizinhança
3. **Features Socioeconômicas**: Índices, indicadores, rankings
4. **Features de Interação**: Combinações entre variáveis
5. **Features de Agregação**: Estatísticas por região e período

## Estrutura do Feature Engineering

- **Seção 1**: Carregamento e preparação dos dados
- **Seção 2**: Features temporais (lags, médias móveis, sazonalidade)
- **Seção 3**: Features espaciais (distâncias, densidades, vizinhança)
- **Seção 4**: Features socioeconômicas (índices, indicadores)
- **Seção 5**: Features de interação e agregação
- **Seção 6**: Seleção e validação de features
- **Seção 7**: Preparação para modelagem


In [ ]:
# ============================================================================
# 1. IMPORTAÇÃO DE BIBLIOTECAS E CONFIGURAÇÃO
# ============================================================================

import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import warnings
from datetime import datetime
import json
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

# Configuração de diretórios
BASE_DIR = Path('.')
DATA_DIR = BASE_DIR / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUT_DIR = BASE_DIR / 'outputs' / 'figures'

# Cria diretório de saída
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📁 Diretório de saída: {OUTPUT_DIR.absolute()}")


## 2. CARREGAMENTO E PREPARAÇÃO DOS DADOS


In [ ]:
# Carrega dados consolidados
df_crimes = pd.read_csv(PROCESSED_DIR / 'crimes_consolidado.csv')
print(f"📊 Dados carregados: {len(df_crimes):,} registros")

# Carrega dados geoespaciais
gdf_crimes = gpd.read_file(PROCESSED_DIR / 'crimes_geo.geojson')
print(f"🗺️ Dados geoespaciais: {len(gdf_crimes):,} registros")

# Converte coluna de data
df_crimes['data'] = pd.to_datetime(df_crimes['data'])

# Informações básicas
print("\n📋 INFORMAÇÕES BÁSICAS:")
print(f"Período: {df_crimes['ano'].min()} - {df_crimes['ano'].max()}")
print(f"Regiões: {df_crimes['regiao_administrativa'].nunique()}")
print(f"Tipos de crime: {df_crimes['tipo_crime'].nunique()}")
print(f"Colunas: {list(df_crimes.columns)}")

# Preview dos dados
print("\n📋 Preview dos dados:")
print(df_crimes.head())


## 3. FEATURES TEMPORAIS


In [ ]:
# Cria features temporais
def create_temporal_features(df):
    """
    Cria features temporais para análise de séries temporais
    """
    df_features = df.copy()
    
    # Features básicas de tempo
    df_features['ano'] = df_features['data'].dt.year
    df_features['mes'] = df_features['data'].dt.month
    df_features['dia'] = df_features['data'].dt.day
    df_features['dia_semana'] = df_features['data'].dt.dayofweek
    df_features['dia_ano'] = df_features['data'].dt.dayofyear
    df_features['trimestre'] = df_features['data'].dt.quarter
    df_features['semestre'] = df_features['data'].dt.month.apply(lambda x: 1 if x <= 6 else 2)
    
    # Features cíclicas
    df_features['mes_sin'] = np.sin(2 * np.pi * df_features['mes'] / 12)
    df_features['mes_cos'] = np.cos(2 * np.pi * df_features['mes'] / 12)
    df_features['dia_semana_sin'] = np.sin(2 * np.pi * df_features['dia_semana'] / 7)
    df_features['dia_semana_cos'] = np.cos(2 * np.pi * df_features['dia_semana'] / 7)
    
    # Features de sazonalidade
    df_features['eh_fim_semana'] = df_features['dia_semana'].isin([5, 6]).astype(int)
    df_features['eh_feriado'] = 0  # Implementar lógica de feriados
    df_features['eh_vacacao'] = df_features['mes'].isin([12, 1, 2, 7]).astype(int)
    
    return df_features

# Aplica features temporais
df_crimes = create_temporal_features(df_crimes)

print("✅ Features temporais criadas")
print(f"📊 Total de colunas: {len(df_crimes.columns)}")

# Mostra novas features
temporal_features = ['ano', 'mes', 'dia', 'dia_semana', 'trimestre', 'semestre', 
                    'mes_sin', 'mes_cos', 'dia_semana_sin', 'dia_semana_cos',
                    'eh_fim_semana', 'eh_vacacao']

print("\n📋 Features temporais criadas:")
for feature in temporal_features:
    if feature in df_crimes.columns:
        print(f"  - {feature}")


In [ ]:
# Cria features de lag e médias móveis
def create_lag_features(df, group_cols, value_col, lags=[1, 3, 6, 12]):
    """
    Cria features de lag e médias móveis
    """
    df_features = df.copy()
    
    # Ordena por grupo e data
    df_features = df_features.sort_values(group_cols + ['data'])
    
    # Cria lags
    for lag in lags:
        df_features[f'{value_col}_lag_{lag}'] = df_features.groupby(group_cols)[value_col].shift(lag)
    
    # Cria médias móveis
    for window in [3, 6, 12]:
        df_features[f'{value_col}_ma_{window}'] = df_features.groupby(group_cols)[value_col].rolling(window=window).mean().reset_index(0, drop=True)
    
    # Cria diferenças
    df_features[f'{value_col}_diff'] = df_features.groupby(group_cols)[value_col].diff()
    df_features[f'{value_col}_diff2'] = df_features.groupby(group_cols)[value_col].diff(2)
    
    # Cria features de tendência
    df_features[f'{value_col}_trend_3m'] = df_features.groupby(group_cols)[value_col].rolling(window=3).apply(lambda x: np.polyfit(range(len(x)), x, 1)[0]).reset_index(0, drop=True)
    
    return df_features

# Aplica features de lag
df_crimes = create_lag_features(df_crimes, 
                               group_cols=['regiao_administrativa', 'tipo_crime'],
                               value_col='total_ocorrencias')

print("✅ Features de lag e médias móveis criadas")

# Mostra features de lag criadas
lag_features = [col for col in df_crimes.columns if 'lag_' in col or 'ma_' in col or 'diff' in col or 'trend' in col]
print(f"\n📋 Features de lag criadas: {len(lag_features)}")
for feature in lag_features[:10]:  # Mostra primeiras 10
    print(f"  - {feature}")


## 4. FEATURES ESPACIAIS
